In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def decode_baby_col(row):    
    row = row.strip()
    parent = row.split(')')
    start = int(parent[0].split('(')[1].strip())
    remaining = ')'.join(parent[1:])
    remaining = remaining.strip(' ')
    find_format = remaining.split('%')
    dtypes_and_name = find_format[0].strip(' ')
    types = [t for t in dtypes_and_name.split(' ') if t]
    assert len(types) == 2, types
    dtype = types[0].strip(' ')
    name = types[1].strip(' ')
    num_c = int(float(find_format[1].split(' ')[0].strip(' ')[:-1]))
    end_of_str = 'f'.join(find_format[1].split('f')[1:])
    end_of_str = end_of_str.strip(' \r\n')        
    return {
        'start': start,
        'end': start + num_c,
        'dtype': dtype,
        'name': name,
        'desc': end_of_str.strip('"')
    }

def read_dct_file(dct):
    """
    read dct file which defines field encoded in main text files
    """
    with open(dct) as r:
        chars_to_tab = r.readlines()
        col_data = [decode_baby_col(c) for c in chars_to_tab if c.startswith('_column')]
        
    col_data_df = pd.DataFrame(col_data)
    start = 1
    columns = []
    column_widths = []
    for i, c in col_data_df.iterrows():
        s = c['start']
        e = c['end']
        if s != start:
            column_widths.append(s - start)
            columns.append('blank{0}'.format(i))
        columns.append(c['name'])
        column_widths.append(e - s)
        start = e
    return col_data_df, columns, column_widths

In [ ]:
def read_row_to_table(rowstr, column_df):
    row = {}
    for idx, data in column_df.iterrows():
        info = rowstr[data['start'] - 1: data['end'] - 1]
        # print(info, data['name'])
        if data['dtype'] == 'int' or data['dtype'] == 'byte':
            if not info.strip():
                info = np.nan
            else:
                info = int(info)
        elif data['dtype'] == 'double':
            info = float(info)
        elif data['dtype'] == 'str1':
            info = str(info)
        else:
            info = str(info)
        row[data['name']] = info
    return row

# def read_nat_data(file):
#     with open(file) as r:
#         for idx, l in enumerate(r):            
#             data = read_row_to_table(l, column_reading)
#             data['idx'] = idx            
#             yield data


# a = read_nat_data('Nat2018PublicUS.c20190509.r20190717.txt')

In [ ]:
# a2 = pd.read_fwf(
#     'Nat2018PublicUS.c20190509.r20190717.txt', 
#     sep='', header=None, chunksize=10000
# )

In [ ]:
# file_as_df = []
# for idx, d in enumerate(a):
#     file_as_df.append(d)
#     if idx % 100000 == 0:
#         print(idx)

In [ ]:
# column_reading = column_reading.sort_values(by=['start'])


In [ ]:
def check_int(x):
    if not isinstance(x, int) and not isinstance(x, float):
        x = int(x) if not x.strip() else np.nan
    return x

def check_double(x):
    if isinstance(x, int):
        x = float(x)
    elif not isinstance(x, float):
        x = float(x) if not x.strip() else np.nan
    return x

def read_data(file, columns, col_widths, col_df, chunk=1000000):
    desired_col = col_df['name'].values
    a2 = pd.read_fwf(
        file, sep='', 
        header=None, chunksize=chunk, widths=column_widths
    )
    groups = col_df.groupby(by=['dtype'])
    
    for df in a2:
        df.columns = columns
        
        for d, grp in groups:
            if d == 'int' or d == 'byte':
                df[grp['name'].values] = df[grp['name'].values].applymap(check_int)
            elif d == 'double':
                df[grp['name'].values] = df[grp['name'].values].applymap(check_double)
        
        yield df[desired_col]
        

In [ ]:
column_df, columns, column_widths = read_dct_file('/mnt/nas/natality/2004/natl2004.dct')
a = read_data('/mnt/nas/natality/2004/Nat2004us.dat', columns, column_widths, column_df, chunk=100)
dfs = []
for i, g in enumerate(a):    
    dfs.append(g)
    if i == 3:
        break
    print(i)

In [ ]:
column_df

In [ ]:
column_df

In [ ]:
dfMerged['educsmsa'].value_counts()

In [ ]:
dfMerged = pd.concat(dfs)
dfMerged['rectype']

In [ ]:
# dfMerged.to_csv('Nat2018PublicUS.c20190509.r20190717.parsed.txt', sep='\t')